In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
torch.cuda.empty_cache()

# **Install dependencies**

In [3]:
%%capture
!pip install tensorflow
!pip install keras
!pip install netron
!pip install nltk
!pip3 install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install nlp
!pip install rouge-score
!pip install git-python
!pip install sentencepiece
!pip install sacrebleu==1.5.1
!pip install tree-sitter
!pip install evaluate
!pip install accelerate>=0.20.1
!pip install transformers[torch]
!pip install --upgrade gradio fastapi typing_extensions
!pip install shap
!pip install parser-libraries
!pip install --upgrade gradio fastapi typing_extensions
!pip install parser-libraries

# **Import Libraries**

In [4]:
import transformers
import datasets
from datasets import load_dataset, load_metric
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

logging.basicConfig(level=logging.INFO)

import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# **cd**

In [6]:
%cd /content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO

/content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO


# **Load Data**

In [7]:
from datasets import load_dataset


### **Python DATASET**

In [8]:
from datasets import load_dataset

raw_dataset = load_dataset("codeparrot/xlcost-text-to-code", "Python-snippet-level")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [9]:
train_dataset= raw_dataset["train"]
val_dataset = raw_dataset["validation"]
test_dataset = raw_dataset["test"]


train_dataset = raw_dataset['train'].select(range(2000))
val_dataset = raw_dataset['validation'].select(range(400))
test_dataset = raw_dataset['test'].select(range(450))


500
62
112


In [10]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'code'],
        num_rows: 81207
    })
    test: Dataset({
        features: ['text', 'code'],
        num_rows: 7293
    })
    validation: Dataset({
        features: ['text', 'code'],
        num_rows: 3946
    })
})

# **Tokenizer**

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-nl", use_fast = True)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [12]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-nl', vocab_size=67028, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	67027: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

# **Model**

In [13]:
#%cd /content/drive/MyDrive/new_models/MarianCG/New_MarianCG/new/test

In [14]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-nl")


pytorch_model.bin:   0%|          | 0.00/316M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [15]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(67028, 512, padding_idx=67027)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(67028, 512, padding_idx=67027)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [16]:
model.config

MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      67027
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 67027,
  "decoder_vocab_size": 67028,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_length": 5

# **Evaluator**

In [17]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO/Utils


In [18]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(tokenizer, device,smooth_bleu=True)

/content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO/Utils/evaluator.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.sacre_bleu: Metric = load_metric('sacrebleu')


In [19]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO




# **Prepare the data.**

In [20]:
encoder_length = 32
decoder_length = 32
batch_size = 1


# map data correctly
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["code"], padding="max_length", truncation=True, max_length=decoder_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch

In [21]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['text', 'code'],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [22]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['text', 'code'],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [23]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=512,padding=True, ####new
                                       model = model)

# **Training**

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./Marian-Training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,

)

In [25]:
#train the model on the original dataset
trainer.train()
trainer.save_model("./Marian-Initial-Training")
trainer.save_state()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,2.513600,1.679595,5.364047,9.802314,16.727845,6.666667,3.757712,20.123674,36.285055,21.294272,12.943820,8.183079
2,1.193100,1.504603,8.641998,14.159756,23.829201,10.647482,6.325301,24.609448,41.462268,28.611580,17.588818,11.782477
3,0.990200,1.463386,10.286627,16.257393,26.432022,12.761355,7.622642,25.325736,43.416091,31.606218,19.880597,13.577185
4,0.885900,1.449833,13.701605,20.549907,33.788707,16.988417,10.369610,26.616576,44.937182,38.909359,25.173745,17.193836
5,0.825000,1.443906,9.801415,15.803522,25.191371,12.290909,7.311500,27.534158,46.816312,30.995204,19.800747,13.212435


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Ther

**Fine Tuning With C**

In [26]:
new_raw_dataset = load_dataset("codeparrot/xlcost-text-to-code", "C++-snippet-level")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [27]:
train_dataset = raw_dataset['train'].select(range(2000))
val_dataset = raw_dataset['validation'].select(range(400))
test_dataset = raw_dataset['test'].select(range(450))

In [28]:
# make train dataset ready
new_train_data = new_train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['text', 'code'],
)
new_train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [29]:
# same for validation dataset
new_val_data = new_val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['text', 'code'],
)
new_val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [30]:
fine_tuning_args = Seq2SeqTrainingArguments(
    output_dir="./Marian-Fine-Tuning-C++",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=5, # Fewer epochs for fine-tuning
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 5e-6, # Lower learning rate
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
)

In [31]:
fine_tuning_trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=fine_tuning_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=new_train_data, # Use the new dataset
    eval_dataset=new_val_data,
)

In [32]:
fine_tuning_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,1.914200,1.734383,3.572883,2.991017,11.387485,4.818797,2.286648,12.467710,23.572931,9.650516,4.193811,1.921470
2,1.220000,1.504514,17.701320,17.119090,30.568256,18.584071,14.712154,24.931402,39.438818,28.475034,18.028169,14.359352
3,1.032500,1.453434,18.511288,17.723861,31.091877,19.305556,15.602322,25.317518,37.674042,28.992461,18.539728,14.981273
4,0.959900,1.438467,21.959039,21.202133,36.611195,23.174603,18.530885,26.293204,38.781115,34.303698,22.415219,18.047079
5,0.919800,1.434273,19.103247,18.192804,31.669993,20.055517,16.243655,26.085648,38.120191,29.525120,19.194824,15.500376


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Ther

TrainOutput(global_step=2500, training_loss=1.209286376953125, metrics={'train_runtime': 398.7365, 'train_samples_per_second': 6.27, 'train_steps_per_second': 6.27, 'total_flos': 21186478080000.0, 'train_loss': 1.209286376953125, 'epoch': 5.0})

In [33]:
fine_tuning_trainer.save_model()
fine_tuning_trainer.save_state()

**Fine Tuning With Java**


In [34]:
java_raw_dataset = load_dataset("codeparrot/xlcost-text-to-code", "Java-snippet-level")

java_train_dataset = java_raw_dataset['train'].select(range(500))
java_val_dataset = java_raw_dataset['validation'].select(range(62))
java_test_dataset = java_raw_dataset['test'].select(range(112))

java_train_data = java_train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['text', 'code'],
)
java_train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

java_val_data = java_val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['text', 'code'],
)
java_val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

java_fine_tuning_args = Seq2SeqTrainingArguments(
    output_dir="./Marian-Fine-Tuning-Java",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=5, # Fewer epochs for fine-tuning
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 5e-6, # Lower learning rate
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
)


java_fine_tuning_trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=java_fine_tuning_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=java_train_data, # Use the new dataset
    eval_dataset=java_val_data,
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [35]:
java_fine_tuning_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,1.469500,1.663412,11.380541,10.343995,24.819143,12.391931,8.786611,16.802078,34.559914,22.491349,11.303828,8.012422
2,1.014300,1.527642,23.905580,22.982391,46.498599,25.668979,19.218585,21.383862,37.103352,43.849206,24.947146,18.552036
3,0.897500,1.487988,18.596493,17.492638,37.045124,20.198171,14.720000,23.733528,40.910867,34.603659,19.360000,13.804714
4,0.840300,1.461897,22.770148,21.425076,43.128781,24.018265,18.393030,24.987560,41.384336,40.493601,23.062016,17.216495
5,0.806700,1.452370,28.693727,26.670608,53.739316,30.205950,23.029557,25.034344,41.863361,51.085714,29.151292,21.837550


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Ther

TrainOutput(global_step=2500, training_loss=1.00565615234375, metrics={'train_runtime': 336.8496, 'train_samples_per_second': 7.422, 'train_steps_per_second': 7.422, 'total_flos': 21186478080000.0, 'train_loss': 1.00565615234375, 'epoch': 5.0})

# **Visualize**

**Marian**

In [36]:
%load_ext tensorboard

In [37]:
pwd

'/content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO'

In [38]:
# tensorboard --logdir ./Marian-Training/runs/

# **Inference**

In [39]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
new_model = AutoModelForSeq2SeqLM.from_pretrained("./Marian-Fine-Tuning-C")
new_model.to("cuda")

new_tokenizer = AutoTokenizer.from_pretrained("./Marian-Fine-Tuning-C", use_fast=True)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [40]:
new_model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(67028, 512, padding_idx=67027)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(67028, 512, padding_idx=67027)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [41]:
new_model.config.max_length


512

# **Evaluator**

In [42]:
from google.colab import files

In [43]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO/Utils


In [44]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(new_tokenizer, device,smooth_bleu=True)

In [45]:
batch_size = 1

def generate_new_code(batch):
    inputs = new_tokenizer(batch["text"], padding="max_length", truncation=True, return_tensors="pt")

    #max_new_tokens=512,

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    output_str = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_code"] = output_str

    return batch

In [46]:
results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size)

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [47]:
# Ok Evaluator
bleu_score =0
rouge_score = 0
sacre_bleu = 0

for i in range(len(results)):
  ref = results["code"][i].strip()
  pred = results["pred_code"][i].replace('▁', ' ').strip()
  if pred is not None and pred != "":
      if ref is not None and ref != "":
        #bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
        #bleu_score  += bleu_metric["score"]

        bleu_metric = evaluator.evaluate([pred], [ref])
        #bleu_metric = evaluator.evaluateSingle(pred, ref)

        #bleu_score += bleu_metric['BLEU-Unigram-Precision']
        bleu_score += bleu_metric['BLEU']
        rouge_score += bleu_metric['ROUGE-L']
        sacre_bleu += bleu_metric['SacreBLEU']
        #print(bleu_metric['SacreBLEU'])
  else:
    continue

# **Performance Metrics**

BLEUScore

In [48]:
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 9.451763862832276 (scale 0-100)


SacreBLEU

In [49]:
print('Sacre Bleu: {} (scale 0-100)'.format(sacre_bleu/len(results)))

Sacre Bleu: 4.903051274159914 (scale 0-100)


ROUGE Score

In [50]:
print('ROUGE Score: {} (scale 0-100)'.format(rouge_score/len(results)))

ROUGE Score: 18.15791538724682 (scale 0-100)


In [51]:
with open("./references.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["text"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i].replace('▁', ' ').strip()

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(ref + '\n')
    else:
      continue


with open("./predictions.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["text"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i].replace('▁', ' ').strip()

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(pred + '\n')
    else:
      continue


code_bleu_score

In [52]:
from google.colab import files

In [53]:
import calc_code_bleu

In [54]:
ngram_match_score, weighted_ngram_match_score, syntax_match_score, dataflow_match_score, code_bleu_score = calc_code_bleu.calc_code_bleu("references.txt","predictions.txt","python", "keywords")

In [55]:
print("ngram_match_score: ", ngram_match_score*100)
print("weighted_ngram_match_score: ", weighted_ngram_match_score*100)
print("syntax_match_score: ", syntax_match_score*100)
print("dataflow_match_score: ", dataflow_match_score*100)
print("code_bleu_score: ", code_bleu_score*100)

ngram_match_score:  1.6598673932817836
weighted_ngram_match_score:  1.7155715267264244
syntax_match_score:  4.142011834319527
dataflow_match_score:  16.50099403578529
code_bleu_score:  6.004611197528256


Exact Match

In [56]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO


In [57]:
pwd

'/content/drive/MyDrive/models/CodeGeneration/MarianCG-DJANGO'

In [58]:
import re
import string

import datasets
import numpy as np

import evaluate
from evaluate import load

In [59]:

exact_match = evaluate.load("exact_match")


In [60]:

#refs = ["The cat sat on the mat.", "Theaters are great.", "It's like comparing oranges and apples."]
#preds = ["The cat sat on the mat?", "Theaters are great.", "It's like comparing apples and oranges."]

"""
for i in range(len(results)):
  refs = results["code"][i]
  preds = results["pred_code"][i]
"""

'\nfor i in range(len(results)):\n  refs = results["code"][i]\n  preds = results["pred_code"][i]\n'

In [61]:
refs = [x.replace('▁', ' ').strip() for x in results["code"]]
preds = [x.replace('▁', ' ').strip() for x in results["pred_code"]]

In [62]:
print(refs[1])
print(preds[1])

mp = { } NEW_LINE sum = 0 NEW_LINE
int frequent ( arr [ ], b [ ], b [ ], b [ i ] ], b [ i ], b [ i ], b [ i ], b [ i ], b [ i ], b [ i ] ++ ) {


In [63]:
#refs = results["code"].strip()
#preds = results["pred_code"].replace('▁', ' ').strip()

In [64]:

exact_match_results = exact_match.compute(references=refs, predictions=preds, regexes_to_ignore=["▁", " "], ignore_case=True, ignore_punctuation=True, ignore_numbers=True)
print("exact_match= ",round(exact_match_results["exact_match"], 5)*100)

exact_match=  0.0


# **Save the  results**

In [65]:
bleu_score = 0
with open("./Marian-Training/Output-Python-C.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["text"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i].replace('▁', ' ').strip()

    if pred is not None and pred != "":
      if ref is not None and ref != "":

        bleu_metric = evaluator.evaluate([pred], [ref])
        f.write( "NL: " + nl + '\n')
        f.write( "ref: " + ref + '\n')
        f.write( "Output: " + pred + '\n')
        f.write("bleu_metric: " + str(bleu_metric["BLEU"]) + '\n')
        f.write(10*"-" + '\n')
        bleu_score  += bleu_metric["BLEU"]

    else:
      continue


In [66]:
test_dataset["code"]

['def minSum ( A , N ) : NEW_LINE',
 'mp = { } NEW_LINE sum = 0 NEW_LINE',
 'for i in range ( N ) : NEW_LINE',
 'sum += A [ i ] NEW_LINE',
 'if A [ i ] in mp : NEW_LINE INDENT mp [ A [ i ] ] += 1 NEW_LINE DEDENT else : NEW_LINE INDENT mp [ A [ i ] ] = 1 NEW_LINE DEDENT',
 "minSum = float ( ' inf ' ) NEW_LINE",
 'for it in mp : NEW_LINE',
 'minSum = min ( minSum , sum - ( it * mp [ it ] ) ) NEW_LINE',
 'return minSum NEW_LINE',
 'arr = [ 4 , 5 , 6 , 6 ] NEW_LINE',
 'N = len ( arr ) NEW_LINE print ( minSum ( arr , N ) ) NEW_LINE',
 'def maxAdjacent ( arr , N ) : NEW_LINE INDENT res = [ ] NEW_LINE DEDENT',
 'for i in range ( 1 , N - 1 ) : NEW_LINE INDENT prev = arr [ 0 ] NEW_LINE DEDENT',
 "maxi = - 1 * float ( ' inf ' ) NEW_LINE",
 'for j in range ( 1 , N ) : NEW_LINE',
 'if ( i == j ) : NEW_LINE INDENT continue NEW_LINE DEDENT',
 'maxi = max ( maxi , abs ( arr [ j ] - prev ) ) NEW_LINE',
 'prev = arr [ j ] NEW_LINE',
 'res . append ( maxi ) NEW_LINE',
 "for x in res : NEW_LINE INDENT pr

# **Test**

In [67]:
# map data correctly
def generate_code(NL):
    inputs = new_tokenizer([NL], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_code = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_code

In [68]:

#NL = "decode a hex string to utf-8."
#NL = "remove first and last lines of string `s`"
NL = "generate mappings for each function node and parameters and variables names associated with it . concode_field_sep int parentScope concode_elem_sep ArrayList functionBracePositions"
output_code = generate_code(NL)

In [69]:
output_code

['for ( int i = 0 ; i < N ; i ++ ) {']

In [70]:
natural_language = "How I can get rid of None values in dictionary?"
gen_code = generate_code(natural_language)

In [71]:
gen_code

['#include <bits/stdc> NEW_LINE using namespace std ;']

In [72]:
# import gradio as gr
# output_text = gr.Textbox()
# gr.Interface(generate_code,"textbox", output_text, title="MarianCG Code Generation",
#              description="MarianCG Code Generation").launch()


In [73]:

# define the input sentences we want to genrate code from
data = [
    "check if all elements in list `myList` are identical"
]


In [74]:
import shap
explainer = shap.Explainer(new_model,new_tokenizer)

shap_values = explainer(data)

shap.plots.text(shap_values)

Output hidden; open in https://colab.research.google.com to view.